# Use Case: ZAF202303 News Classifier


### Overview
(example) We are building a web application with API access to a machine learning model without giving source of code or any details how it works. We are interested in building a high quality text classifier that categorizes new stories into 2 categories - information and entertainment. We want the classifier to stick with predicting the better among these two categories (this classifier will not predict a percentage score for these two categories).


### Methodology
(describe the steps)
There are 10000 different news stories and additional news stories.
- Run the classifier on 1 mln random stories out of 1000 news sources. Get 10k stories where the classifier output is the closest to the decision boundary and get these examples labeled.
- Get 10k random labeled stories from the 10k random labeled stories from 1000 news we care about.
- Pick a random sample of 1 mln stories from 1000 news sources and have them labeled. Pick the subset of 10k stories where the model’s output is both wrong and farthest away from the decision boundary.

Using different methods of classifying a bag of news articles from 1000 news sources, measure the accuracy of the model with an original train dataset.

### Business Segments
(advise for which business segments you can apply the model)
Business Segments are the following, please following the names of business segments given:

- Airlines
- Autonomous Driving
- Banking & Finance
- Business & Private Sector
- Government & Public Services
- Healthcare
- Human Resources
- Lifestyle & Social Media
- Media & Publishing
- Retail & E-commerce


### AI Application
(describe how your model can be useful in real life or business)
- Objective
- Models
- Predictions format


### Data
Put a link to the dataset.


### Papers
Advise the papers you referred to for citation.


### Demo link
Record a 1 min demo and kindly provide a link for this.


### Team
Put the names of all contributors and their contacts, Github names if you want.

## 1 - Load the data

In [1]:
# your code is here

## 2 - Data Preprocessing, Exploratory Data Analysis

In [11]:
# your code is here

## 3 - Models
Write a code for training, validating, testing for all the models you are capable of in the notebook `model.ipynb`. Put as much comments as you can what you do, how the pre-trained model works, which issues you face. We want to deploy your model into production, please help us to do so by clearly saying what you do and why it is good.

**Write a clear conclusion which model can be recommended to the client at the of this part.**

In [12]:
# your code is here

### *Conclusion:* The model which can be recommended to the client is ... (Put here also essential recommendations to the client how to use your model).

## 4 - Saving the model
Save the DS best model in the Jupyter notebook `model.ipynb` in one of the following formats:

- `network.save('model.h5')` #keras
- `joblib.dump(model, "model.pkl")` # optional
- `torch.save(model.state_dict(), './model.pt')` #pytorch
- `model.save('path/to/model')`

End of document.